In [ ]:
import deepxde as dde
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from naca4digit_airfoil import Naca4DigitAirfoil
# from navier_stokes_with_data import NavierStokesPDEs
from navier_stokes import NavierStokesPDEs
import utils
import pandas as pd
from scipy.interpolate import griddata

In [ ]:
cfd_df = pd.read_csv('NACA2412.txt', delimiter=',')

In [ ]:
x_min = -5
x_max = 15
y_min = -5
y_max = 5

In [ ]:
dde.config.set_random_seed(48)
dde.config.set_default_float('float64')
dde.config.set_parallel_scaling('strong')

rho  = 1.225
mu   = 1.789e-5
# u_in  = 15
u_inlet  = 1

In [ ]:
rho, mu

In [ ]:
L = 1
airfoil = Naca4DigitAirfoil(c=L, P=4, T=12, a=0, offset_x=0, offset_y=0)

In [ ]:
Re = rho * u_inlet * L / mu

In [ ]:
# Geometry defintion
farfield = dde.geometry.Rectangle([x_min, y_min], [x_max, y_max])
airfoil_geom  = dde.geometry.Polygon(airfoil.get_boundary_points(250))
geom     = dde.geometry.CSGDifference(farfield, airfoil_geom)

inner_rec  = dde.geometry.Rectangle([-0.5, -0.5], [0.5, 0.5])
inner_dom  = dde.geometry.CSGDifference(inner_rec, airfoil_geom)

outer_dom  = dde.geometry.CSGDifference(farfield, inner_rec)
outer_dom  = dde.geometry.CSGDifference(outer_dom, airfoil_geom)

In [ ]:
# Nf1 = 2**15 # = 32768
# Nf2 = 2**16 # = 65536
# Nb  = 2**11 # = 2048
Nf1 = 2**10
Nf2 = 2**10
Nb  = 2**10
Ns  = 250

random = "Sobol"
inner_points = inner_dom.random_points(Nf1, random=random)
outer_points = outer_dom.random_points(Nf2, random=random)

farfield_points = farfield.random_boundary_points(Nb, random=random)
airfoil_points  = airfoil.get_boundary_points(Ns)

points = np.append(inner_points, outer_points, axis = 0)
points = np.append(points, airfoil_points, axis = 0)

In [ ]:
fig1, ax1 = plt.subplots(figsize = (16, 9))
plt.axis('equal')
plt.scatter(farfield_points[:, 0], farfield_points[:, 1], s = 0.5, c='#40798C')  # Ocean blue for farfield points
plt.scatter(inner_points[:, 0], inner_points[:, 1], s = 0.5, c = '#F6AE2D')  # Sunflower yellow for inner points
plt.scatter(outer_points[:, 0], outer_points[:, 1], s = 0.5, c = '#F6AE2D')  # Sunflower yellow for outer points
plt.scatter(airfoil_points[:, 0], airfoil_points[:, 1], s = 0.5, c = '#E9897E')  # Sunset coral for airfoil points
plt.fill(airfoil_points[:, 0], airfoil_points[:, 1])
plt.xlim([x_min - 0.1, x_max + 0.1])
plt.ylim([y_min - 0.1, y_max + 0.1])

In [ ]:
X, Y = np.meshgrid(np.linspace(cfd_df['x'].min(), cfd_df['x'].max(), 1000),
                             np.linspace(cfd_df['y'].min(), cfd_df['y'].max(), 1000))

U = griddata((cfd_df['x'], cfd_df['y']), cfd_df['u'], (X, Y), method='linear')

fig, ax = plt.subplots(figsize=(10, 6))

clev = np.arange(U.min(), U.max(), 0.01)
cp = ax.contourf(X, Y, U, clev, cmap='jet')
fig.colorbar(cp, ax=ax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Velocity Field - X Component')
airfoil.plot(ax)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plt.show()

In [ ]:
V = griddata((cfd_df['x'], cfd_df['y']), cfd_df['v'], (X, Y), method='linear')

fig, ax = plt.subplots(figsize=(10, 6))

clev = np.arange(V.min(), V.max(), 0.01)
cp = ax.contourf(X, Y, V, clev, cmap='jet')
fig.colorbar(cp, ax=ax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Velocity Field - Y Component')
airfoil.plot(ax)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plt.show()

In [ ]:
P = griddata((cfd_df['x'], cfd_df['y']), cfd_df['p'], (X, Y), method='linear')

fig, ax = plt.subplots(figsize=(10, 6))

clev = np.arange(P.min(), P.max(), 1)
cp = ax.contourf(X, Y, P, clev, cmap='jet')
fig.colorbar(cp, ax=ax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Pressure Field')
airfoil.plot(ax)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plt.show()

In [ ]:
# x_data = farfield_points[:, 0]
# y_data = farfield_points[:, 1]

# grid_points = cfd_df[['x', 'y']].values
# u_values = cfd_df['u'].values / sf_uv
# v_values = cfd_df['v'].values / sf_uv
# p_values = cfd_df['p'].values

# u_interp = griddata(grid_points, u_values, (x_data, y_data), method='linear')
# v_interp = griddata(grid_points, v_values, (x_data, y_data), method='linear')
# p_interp = griddata(grid_points, p_values, (x_data, y_data), method='linear')

# uvp_data = np.vstack((u_interp, v_interp, p_interp)).T
# # uv_data = np.vstack((u_interp, v_interp)).T

# u_data = np.array([uvp_data[i][0] for i in range(len(uvp_data))])
# v_data = np.array([uvp_data[i][1] for i in range(len(uvp_data))])
# p_data = np.array([uvp_data[i][2] for i in range(len(uvp_data))])

# # u_data = np.array([uv_data[i][0] for i in range(len(uv_data))])
# # v_data = np.array([uv_data[i][1] for i in range(len(uv_data))])

In [ ]:
# fig1, ax1 = plt.subplots(figsize = (16, 9))
# plt.axis('equal')
# scatter = plt.scatter(x_data, y_data, c=u_data, s=0.5, cmap='jet')
# plt.colorbar(scatter)
# plt.title('Velocity Field - X Component')
# plt.scatter(airfoil_points[:, 0], airfoil_points[:, 1], s = 0.5, c = '#E9897E')  # Sunset coral for airfoil points
# plt.fill(airfoil_points[:, 0], airfoil_points[:, 1])
# plt.xlim([x_min_scaled - 0.1, x_max_scaled + 0.1])
# plt.ylim([y_min_scaled - 0.1, y_max_scaled + 0.1])
# plt.show()

In [ ]:
# fig1, ax1 = plt.subplots(figsize = (16, 9))
# plt.axis('equal')
# scatter = plt.scatter(x_data, y_data, c=v_data, s=0.5, cmap='jet')
# plt.colorbar(scatter)
# plt.title('Velocity Field - Y Component')
# plt.scatter(airfoil_points[:, 0], airfoil_points[:, 1], s = 0.5, c = '#E9897E')  # Sunset coral for airfoil points
# plt.fill(airfoil_points[:, 0], airfoil_points[:, 1])
# plt.xlim([x_min_scaled - 0.1, x_max_scaled + 0.1])
# plt.ylim([y_min_scaled - 0.1, y_max_scaled + 0.1])
# plt.show()

In [ ]:
# fig1, ax1 = plt.subplots(figsize = (16, 9))
# plt.axis('equal')
# scatter = plt.scatter(x_data, y_data, c=p_data, s=0.5, cmap='jet')
# plt.colorbar(scatter)
# plt.title('Pressure Field')
# plt.scatter(airfoil_points[:, 0], airfoil_points[:, 1], s = 0.5, c = '#E9897E')  # Sunset coral for airfoil points
# plt.fill(airfoil_points[:, 0], airfoil_points[:, 1])
# plt.xlim([x_min_scaled - 0.1, x_max_scaled + 0.1])
# plt.ylim([y_min_scaled - 0.1, y_max_scaled + 0.1])
# plt.show()

In [ ]:
navier_stokes_pdes = NavierStokesPDEs(rho=rho, mu=mu,
                                      xmin=x_min, xmax=x_max, ymin=y_min, ymax=y_max,
                                      airfoil_geom=airfoil_geom, geom=geom,
                                      u_inlet=u_inlet, v_farfield=0)

pdes_fun = navier_stokes_pdes.get_pdes
bcs = navier_stokes_pdes.get_bcs()

In [ ]:
# Problem setup
data = dde.data.PDE(geom, pdes_fun, bcs, num_domain = 0, num_boundary = 0, num_test = 5000, anchors = points)

dde.config.set_default_float('float64')

In [ ]:
plt.figure(figsize = (32, 18))
plt.scatter(data.train_x_all[:,0], data.train_x_all[:,1], s = 0.3)
plt.axis('equal')

In [ ]:
# Neural network definition
# layer_size  = [2] + [32] * 10 + [2]
layer_size  = [2] + [32] * 10 + [3]
# layer_size  = [2] + [32] * 10 + [5]
# activation  = 'silu'
activation  = 'tanh' 
# n           = 10
# activation  = f"LAAF-{n} tanh"
initializer = 'Glorot uniform'

net = dde.nn.FNN(layer_size, activation, initializer)

# Model definition
model = dde.Model(data, net)

In [ ]:
name_generator = utils.NameGenerator()
model_name = name_generator.generate_name()
model_name

In [ ]:
# model.compile(optimizer = 'adam', lr = 5e-4, loss_weights=[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])
model.compile(optimizer = 'adam', lr = 5e-4, loss_weights=[1, 1, 2, 2, 2, 2, 2])
# model.compile(optimizer = 'adam', lr = 1e-3, loss_weights=[1, 1, 2, 2, 2, 2, 2])

In [ ]:
try:
  losshistory, train_state = model.train(epochs = 1000, display_every = 1, model_save_path = './' + model_name + '/')
  dde.saveplot(losshistory, train_state, issave = True, isplot = True)

except KeyboardInterrupt:
  print(f"Training stopped by user.")
  print("=======================================================")

In [ ]:
# model_name = 'AtmosElegantGalaxy375'
# model.restore('./out/AtmosElegantGalaxy375/-20000.ckpt.index')

In [ ]:
# Plotting tool: thanks to @q769855234 code snippet
dx = 0.01
dy = 0.01
x = np.arange(x_min, x_max + dy, dx)
y = np.arange(y_min, y_max + dy, dy)

X = np.zeros((len(x)*len(y), 2))
xs = np.vstack((x,)*len(y)).reshape(-1)
ys = np.vstack((y,)*len(x)).T.reshape(-1)
X[:, 0] = xs
X[:, 1] = ys

In [ ]:
# Model predictions generation
u = model.predict(X, operator = navier_stokes_pdes.getU)
v = model.predict(X, operator = navier_stokes_pdes.getV)
p = model.predict(X, operator = navier_stokes_pdes.getP)

for i in range(len(X)):
   if airfoil_geom.inside(np.array([X[i]]))[0]:
       u[i] = 0.0
       v[i] = 0.0

u = u.reshape(len(y), len(x))
v = v.reshape(len(y), len(x))
p = p.reshape(len(y), len(x))

airfoil_plot = airfoil.get_boundary_points(150)

fig2, ax2 = plt.subplots(figsize = (16, 9))
ax2.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(u.min(), u.max(), 0.1)
cnt2 = ax2.contourf(x, y, u, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig2.colorbar(cnt2)
plt.savefig('./u.png')

fig3, ax3 = plt.subplots(figsize = (16, 9))
ax3.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(v.min(), v.max(), 0.01)
cnt3 = ax3.contourf(x, y, v, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig3.colorbar(cnt3)
plt.savefig('./v.png')

fig1, ax1 = plt.subplots(figsize = (16, 9))
#ax1.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(p.min(), p.max(), 1)
cnt1 = ax1.contourf(x, y, p, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig1.colorbar(cnt1)
plt.savefig('./p.png')